# Creating grid to municipality matrix

## Using number of buildings
The files were saved in the notebook `02.0_preprocessing.ipynb`

In [ ]:
%load_ext jupyter_black
import pandas as pd
import numpy as np
import os
from pathlib import Path
import geopandas as gpd

In [ ]:
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/input/"
)
output_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/02_housing_damage/output/"
)

In [ ]:
grid_land_overlap = gpd.read_file(
    output_dir / "phl_0.1_degree_grid_land_overlap.gpkg"
)
adm3_shp = gpd.read_file(input_dir / "phl_adminboundaries_candidate_adm3.zip")

In [ ]:
# changing this to first check if buildings are in a municipality and filtering those out before joining with grid.
phl_bld_centroids = gpd.read_file(input_dir / "phl_bld_centroids.gpkg")

In [ ]:
phl_bld_centroids_within = gpd.sjoin(
    phl_bld_centroids, adm3_shp, how="inner", predicate="within"
)

In [ ]:
phl_bld_centroids_within.drop(["index_right"], axis=1, inplace=True)

In [ ]:
phl_bld_centroid_grid_count = grid_land_overlap.merge(
    grid_land_overlap.sjoin(phl_bld_centroids_within, how="left")
    .groupby("id")
    .count()
    .geometry.rename("numbuildings")
    .reset_index()
)

In [ ]:
# reading intersection csv
phl_bld_centroid_municip_intersection_count = pd.read_csv(
    input_dir / "phl_hotosm_bld_centroid_municip_intersection_count.csv"
)
# phl_bld_centroid_grid_count = pd.read_csv(input_dir / "phl_hotosm_bld_centroid_grid_count.csv")

In [ ]:
phl_bld_centroid_grid_count

In [ ]:
phl_bld_grid_matrix = phl_bld_centroid_municip_intersection_count.merge(
    phl_bld_centroid_grid_count, on="id", suffixes=("_x", None)
)
phl_bld_grid_matrix.columns

In [ ]:
phl_bld_grid_matrix["weight"] = np.where(
    phl_bld_grid_matrix["numbuildings"] == 0,
    0,
    phl_bld_grid_matrix["numbuildings_x"]
    / phl_bld_grid_matrix["numbuildings"],
)
phl_bld_grid_matrix["weight"].describe()

In [ ]:
phl_bld_grid_matrix[phl_bld_grid_matrix["Centroid_x"] == "126.4E_8.4N"]

In [ ]:
phl_bld_grid_matrix_sel = phl_bld_grid_matrix[
    ["ADM3_PCODE", "ADM3_EN", "id", "Centroid", "weight"]
]
phl_bld_grid_matrix_sel

In [ ]:
phl_bld_grid_matrix_sel.to_csv(
    input_dir / "phl_bld_grid_to_municip_matrix.csv", index=False
)